# Genetic algo for lighthouse sensor distribution on arbitrary mesh

In [1]:
import numpy as np
from stl import mesh as meshstl
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from plotmesh import plot_mesh
import math
import random
from pyquaternion import Quaternion
from scipy.linalg import qr
import roslib
import rospy
import math
import tf

In [2]:
stl_file = 'roboy_models/TestCube/stls/sphere224.stl'

In [3]:
#Move Lighthouses to
translationLH1 = [-2.,0,2.]
quat1 = Quaternion(axis=[0,0,1],angle=-np.pi / 2)

global LH1 
LH1 = (translationLH1, quat1)

translationLH2 = [2,0.,2.]
quat2 = Quaternion(axis=[0,0,1], angle=np.pi / 2)

global LH2
LH2 = (translationLH2, quat2)

print(LH1); print(LH2)

print(quat1)

([-2.0, 0, 2.0], Quaternion(0.70710678118654757, -0.0, -0.0, -0.70710678118654746))
([2, 0.0, 2.0], Quaternion(0.70710678118654757, 0.0, 0.0, 0.70710678118654746))
0.707 -0.000i -0.000j -0.707k


In [6]:
from visualization_msgs.msg import Marker, MarkerArray
from data.trafomatrix import get_random_trafomatrix

scale = 0.01
duration = 1
pos = [0,0,0]
orientationMesh = Quaternion(axis=(1,0,0),angle = np.pi*0)

rospy.init_node('fixed_tf_broadcaster')

#Visualization mesh
topic = 'visualization_marker'
publisher = rospy.Publisher(topic, Marker, queue_size=1)

rate = rospy.Rate(10.0)
rate2 = rospy.Rate(1)

In [12]:
while not rospy.is_shutdown():
    
    for i in range(3):
        
        br = tf.TransformBroadcaster()


        br.sendTransform((LH1[0][0], LH1[0][1], LH1[0][2]),
                         (quat1[0], quat1[1], quat1[2], quat1[3]),
                         rospy.Time.now(),
                         "lighthouse1",
                         "world")
        rate.sleep()
        br.sendTransform((LH2[0][0], LH2[0][1], LH2[0][2]),
                         (quat2[0], quat2[1], quat2[2], quat2[3]),
                         rospy.Time.now(),
                         "lighthouse2",
                         "world")

        rate.sleep()
        
        mesh = Marker()
        mesh.header.frame_id = 'world';
        mesh.ns = 'ns';
        mesh.type = Marker.MESH_RESOURCE
        mesh.color.r = 1.0
        mesh.color.g = 1.0
        mesh.color.b = 1.0
        mesh.color.a = 0.9
        mesh.scale.x = scale
        mesh.scale.y = scale
        mesh.scale.z = scale
        mesh.lifetime = rospy.Duration(-1);
        mesh.header.stamp = rospy.Time.now()
        mesh.action = Marker.ADD
        mesh.id = 10000000;
        mesh.pose.position.x = pos[0];
        mesh.pose.position.y = pos[1];
        mesh.pose.position.z = pos[2];
        mesh.pose.orientation.x = orientationMesh[1];
        mesh.pose.orientation.y = orientationMesh[2];
        mesh.pose.orientation.z = orientationMesh[3];
        mesh.pose.orientation.w = orientationMesh[0];
        mesh.mesh_resource = ("package://" + stl_file)
        #mesh.mesh_resource = "package://roboy_models/roboy_2_0_simplified/meshes/CAD/head.stl"

        publisher.publish(mesh);
        rate2.sleep()
        #MeshMarker.type = visualization_msgs::Marker::MESH_RESOURCE;
        #MeshMarker.mesh_resource = "package://pr2_description/meshes/base_v0/base.dae";
    break
    #print(mesh)

# Preprocess data

In [11]:
#Get mesh vertices and normals
mesh1 = meshstl.Mesh.from_file(('../src/'+ stl_file))
#mesh1 = meshstl.Mesh.from_file('../src/roboy_models/roboy_2_0_simplified/meshes/CAD/torso.stl')

global triangles 
triangles = scale * np.matrix(mesh1.points)


vertices = np.reshape(triangles,(len(triangles)*3,3)) 

#normals = np.zeros_like((len(triangles),3))
normalsNotNorm = mesh1.normals

normals = []
for normal in normalsNotNorm:
    normals.append(1/np.linalg.norm(normal,2)*normal)
    
normals = np.matrix(normals)
normals = scale * normals

#Initialize sensors in centers of triangle
sensors = (triangles[:,0:3]+triangles[:,3:6]+triangles[:,6:9])/3

print('%d triangles' %len(triangles))
#print(triangles)
print('')
print('%d vertices' %len(vertices))
#print(vertices)
print('')
print('%d normals normiert' %len(normals))
#print(normals)
print('')
print('%d sensors in centers of triangles' %len(sensors))
#print(sensors)
#print(vertices); print(normals); print(sensors)

224 triangles

672 vertices

224 normals normiert

224 sensors in centers of triangles


In [13]:
#Prepare vertices, normals and sensor locations for Transformation

verticesRot = np.insert(vertices, 3,values=1,axis=1)
print(verticesRot)
print('')

normalsRot = np.insert(normals, 3, values=0, axis=1)
print(normalsRot)
print('')

sensorsRot = np.insert(vertices, 3,values=1,axis=1)
print(sensorsRot)
print('')

[[ -5.72377014e-02   2.42112732e-02  -1.16002989e-01   1.00000000e+00]
 [ -2.30117798e-02  -7.60902405e-03  -1.34323759e-01   1.00000000e+00]
 [ -7.21518707e-02  -1.52587891e-07  -1.07804518e-01   1.00000000e+00]
 ..., 
 [ -3.49208832e-03  -9.99381924e-02  -2.70083880e-02   1.00000000e+00]
 [ -6.60676956e-04  -9.94094467e-02  -4.85675335e-02   1.00000000e+00]
 [  2.06223106e-02  -9.85083485e-02  -4.05343151e-02   1.00000000e+00]]

[[ -4.75511312e-03  -4.96887323e-05  -8.79695654e-03   0.00000000e+00]
 [ -9.87434804e-03   3.65564600e-04   1.53740928e-03   0.00000000e+00]
 [  3.12507264e-04   7.22206086e-04   9.96899009e-03   0.00000000e+00]
 [  9.87468779e-03   3.26725803e-04  -1.54395431e-03   0.00000000e+00]
 [  4.76005256e-03   1.40244782e-03   8.68187964e-03   0.00000000e+00]
 [  5.89473426e-03   7.55205974e-04   8.04249942e-03   0.00000000e+00]
 [  8.21867645e-03   1.42911509e-03  -5.51461637e-03   0.00000000e+00]
 [  2.63164967e-03   1.03196383e-03  -9.59215641e-03   0.00000000e+0

In [14]:
#Test transform vertices
T = get_random_trafomatrix()
newvertices = np.dot(T,np.transpose(verticesRot)).T


#print(newvertices)

In [15]:
#Sensor visualization in RVIZ


for i,sensor in enumerate(sensors):

    sensor = np.squeeze(np.array(sensor))
    
    rate3 = rospy.Rate(10)
    while not rospy.is_shutdown():

        markers = Marker()
        markers.header.frame_id = 'world'
        markers.ns = 'ns'
        markers.type = Marker.SPHERE

        markers.pose.position.x = sensor[0];
        markers.pose.position.y = sensor[1];
        markers.pose.position.z = sensor[2];
        markers.pose.orientation.x = 0
        markers.pose.orientation.y = 0
        markers.pose.orientation.z = 0
        markers.pose.orientation.w = 0

        #print(marker.points)
        markers.color.r = 0.0
        markers.color.g = 0.0
        markers.color.b = 1.0
        markers.color.a = 1.0

        markers.scale.x = 0.01
        markers.scale.y = 0.01
        markers.scale.z = 0.01
        markers.lifetime = rospy.Duration(-1);
        markers.header.stamp = rospy.Time.now()
        markers.action = Marker.ADD
        markers.id = 100+i

        #mesh.mesh_resource = "package://common_utilities/stls/testcube_40mm.stl"
        #mesh.mesh_resource = "package://roboy_models/roboy_2_0_simplified/meshes/CAD/head.stl"

        publisher.publish(markers);
        rate3.sleep()
        break


# GA

In [16]:
from deap import algorithms, base, creator, tools

In [17]:
#sensors to dict
sensorList = []

for sensor in sensors:
    sensor = np.squeeze(np.array(sensor)).tolist()
    sensorList.append(sensor)

sensor_dict =  list(zip(range(len(sensors)), sensorList))

In [18]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,1.0,1.0)) # 1 -> maximum probblem
creator.create("Individual", list, fitness=creator.FitnessMax)

In [19]:
toolbox = base.Toolbox()

In [20]:
# Find random sensors
def randomSensor(v):
    i = random.randint(0,len(v)-1)
    #print(i)
    rand = v[i]
    
    return rand[0]

In [21]:
r = np.squeeze(np.array(randomSensor(sensors)))

print(r)

[ 0.01959535 -0.03528325  0.05385132]


In [22]:
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", randomSensor, sensor_dict)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 8)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [23]:
toolbox.attr_bool()

67

In [24]:
toolbox.individual()

[220, 142, 47, 69, 11, 93, 223, 66]

In [25]:
toolbox.population(n=30)

[[207, 155, 190, 158, 97, 22, 221, 186],
 [79, 51, 58, 117, 72, 36, 24, 33],
 [20, 68, 182, 168, 170, 133, 120, 29],
 [9, 181, 22, 70, 112, 127, 119, 118],
 [70, 104, 140, 124, 102, 103, 173, 96],
 [90, 113, 103, 51, 142, 120, 11, 160],
 [182, 30, 165, 108, 215, 134, 96, 33],
 [2, 139, 65, 185, 76, 166, 39, 60],
 [108, 24, 44, 13, 97, 178, 152, 68],
 [58, 21, 33, 20, 208, 62, 104, 46],
 [125, 109, 184, 165, 124, 87, 129, 84],
 [63, 45, 34, 200, 149, 222, 56, 2],
 [180, 209, 16, 46, 92, 107, 145, 188],
 [170, 210, 67, 179, 33, 202, 76, 168],
 [144, 213, 170, 207, 157, 177, 78, 185],
 [18, 108, 81, 185, 61, 10, 51, 16],
 [11, 211, 198, 58, 172, 50, 52, 1],
 [209, 81, 153, 4, 89, 132, 48, 194],
 [70, 113, 11, 148, 192, 174, 189, 51],
 [157, 200, 183, 56, 183, 100, 24, 169],
 [83, 182, 80, 97, 182, 10, 72, 211],
 [99, 58, 103, 131, 188, 66, 76, 81],
 [104, 80, 110, 58, 121, 153, 181, 172],
 [99, 53, 217, 18, 139, 75, 101, 119],
 [144, 188, 152, 194, 206, 195, 128, 40],
 [93, 99, 15, 142, 2

# Evaluation (Fitness) Function

In [26]:
from data.RayIntersectsTriangle import rayIntersectsTriangle,rayIntersectsTriangleVisualization

def FitnessFunction(sensors):
    
    #print(sensors)
    #print(sensor_dict)
    
    #1. COMPUTE VISIBLE SENSORS AT THE MOMENT
    LH1_array = np.asarray(LH1[0])
    LH2_array = np.asarray(LH2[0])
    #testTriangle = np.squeeze(np.asarray(triangles[0]))

    visibleLH1 = 0
    visibleLH2 = 0

    for i, nmbr_sensor in enumerate(sensors):
        #print(nmbr_sensor)
        sensor = sensor_dict[nmbr_sensor][1]
        #print(sensor)
        #get distance of current sensor and check if intersection
        interDistLH1 = rayIntersectsTriangle(LH1_array, sensor, 
                                             np.squeeze(np.asarray(triangles[i])), 'lighthouse1')
        interDistLH2 = rayIntersectsTriangle(LH2_array, sensor, 
                                             np.squeeze(np.asarray(triangles[i])), 'lighthouse2')
        #print(interDistLH1)
        #print(interDistLH2)

        for j in range(len(triangles)):
            if(i != j):
                #print(j)
                newInterDistLH1 = rayIntersectsTriangle(LH1_array, sensor, 
                                                        np.squeeze(np.asarray(triangles[j])), 'lighthouse1')
                newInterDistLH2 = rayIntersectsTriangle(LH2_array, sensor, 
                                                        np.squeeze(np.asarray(triangles[j])), 'lighthouse2')
                if(newInterDistLH1 > interDistLH1 and newInterDistLH1 != False):
                    interDistLH1 = newInterDistLH1
                    visibleLH1 += 1

                if(newInterDistLH2 > interDistLH2 and newInterDistLH2 != False):
                    interDistLH2 = newInterDistLH2
                    visibleLH2 += 1
    
    #2. COMPUTE EUCLIDEAN DISTANCE OF SENSORS
    individual = sensors
    dist = 0
    for i,ind in enumerate(individual):
        ind = np.asarray(sensor_dict[ind][1])
        for j in range(i,len(individual)):
            if(i != j):
                indivi = np.asarray(sensor_dict[individual[j]][1])
                dist += np.linalg.norm(ind-indivi)
    
    #print(visibleLH1);print(visibleLH2);print('')
    
    return dist, visibleLH1, visibleLH2

In [27]:
toolbox.register("evaluate", FitnessFunction)

toolbox.register("mate", tools.cxTwoPoint)
# Independent probability  : for each attribute to be mutated.# low~up rondom int
toolbox.register("mutate", tools.mutUniformInt, low=0, up=len(sensorList)-1, indpb=0.2) 
toolbox.register("select", tools.selTournament, tournsize=3)

In [28]:
# Creating population

population = toolbox.population(n=10)

In [29]:
hof = tools.HallOfFame(1)

In [30]:
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [31]:
population, log = algorithms.eaSimple(population, 
                                      toolbox, 
                                      cxpb=0.5, 
                                      mutpb=0.2, 
                                      ngen=1000, 
                                      stats=stats, 
                                      halloffame=hof, 
                                      verbose=True)

gen	nevals	avg    	std    	min    	max
0  	10    	9.76279	4.36544	3.14608	14 
1  	4     	9.5039 	4.12672	3.5003 	14 
2  	9     	9.54814	4.1513 	3.52724	15 
3  	9     	9.36229	3.93138	3.57215	13 
4  	4     	9.32415	3.88564	3.60492	13 
5  	8     	9.30737	3.86065	3.59598	13 
6  	5     	9.22003	3.77517	3.90884	13 
7  	4     	9.28418	3.79191	3.86222	13 
8  	9     	9.33864	3.86484	3.68643	13 
9  	5     	9.34843	3.82496	3.7304 	13 
10 	6     	9.39044	3.83972	3.96918	13 
11 	6     	9.45627	3.91104	3.92146	14 
12 	7     	9.69473	4.09112	3.92077	14 
13 	8     	9.93486	4.20053	3.98168	13 
14 	5     	9.9087 	4.16988	3.98918	13 
15 	8     	9.96257	4.25255	3.78402	14 
16 	6     	9.94132	4.21532	3.88307	14 


KeyboardInterrupt: 

In [ ]:
best_sensors = tools.selBest(population, k=1)[0]
print(best_sensors)

In [ ]:
#Sensor visualization in RVIZ

for i, best in enumerate(best_sensors):

    show_sensor = sensor_dict[best][1]
    
    
    
    rate3 = rospy.Rate(10)
    while not rospy.is_shutdown():

        markers = Marker()
        markers.header.frame_id = 'world'
        markers.ns = 'ns'
        markers.type = Marker.SPHERE

        markers.pose.position.x = show_sensor[0];
        markers.pose.position.y = show_sensor[1];
        markers.pose.position.z = show_sensor[2];
        markers.pose.orientation.x = 0
        markers.pose.orientation.y = 0
        markers.pose.orientation.z = 0
        markers.pose.orientation.w = 0

        #print(marker.points)
        markers.color.r = 1.0
        markers.color.g = 0.0
        markers.color.b = 0.0
        markers.color.a = 1.0

        markers.scale.x = 0.015
        markers.scale.y = 0.015
        markers.scale.z = 0.015
        markers.lifetime = rospy.Duration(-1);
        markers.header.stamp = rospy.Time.now()
        markers.action = Marker.ADD
        markers.id = 100+i

        #mesh.mesh_resource = "package://common_utilities/stls/testcube_40mm.stl"
        #mesh.mesh_resource = "package://roboy_models/roboy_2_0_simplified/meshes/CAD/head.stl"

        publisher.publish(markers);
        rate3.sleep()
        break
